In [42]:
import math
from string import Template

* **Total NAV in EUR**

  * Take this from the Net Asset Value summary in the broker report.
  * Use the total NAV in EUR for the reporting date (e.g. 16.10.2025).
  * This is the denominator for all portfolio percentages.

* **USD equity value in EUR**

  * From the positions / mark-to-market section of the report:
    * Find the block with USD-denominated equities.
    * Use the current "Total in EUR" for that block (i.e. the EUR equivalent of all USD stocks).
  * We ignore USD cash here and focus on USD equity exposure.

This tells us how much of the portfolio is effectively “in USD”.

* **Annualized FX volatility of EUR/USD**

  * This is the annualized volatility of EUR/USD (in decimal form).
    * For example, 5% vol ⇒ `0.05`
  * You can compute this with:
    1. Daily EUR/USD close prices over some window (e.g. last 1 year).
    2. Daily log returns: `r_t = ln(S_t / S_{t-1})`
    3. Standard deviation of daily returns: `sigma_daily`
    4. Annualized vol: `sigma_ann = sigma_daily * sqrt(252)`


In [43]:
# __INPUT__

# fill these 3 numbers each month from your broker statement + FX analysis.

# (a) Total NAV in EUR
nav_eur = 7551.24   # <-- CHANGE THIS EVERY MONTH

# (b) USD equity value in EUR (only USD stocks, no cash)
usd_equity_value_eur = 3762.18   # <-- CHANGE THIS EVERY MONTH

# (c) Annualized FX volatility (EUR/USD, log-returns)
ann_fx_vol = 0.05   # <-- CHANGE THIS WHEN YOUR VOL ESTIMATE CHANGES

# (*) confidence level and horizon
z_95 = 1.645         # 95% one-sided quantile
months_horizon = 1   # 1-month horizon

In [49]:
# __CALCULATIONS__

# Net USD equity exposure as share of NAV
W_usd = usd_equity_value_eur / nav_eur  

# Convert annualized FX vol to 1-month vol
sigma_1m = ann_fx_vol * math.sqrt(months_horizon / 12)

# Parametric 1M FX VaR (95%) in EUR:
# VaR ≈ z * W_usd * NAV * sigma_1m
fx_var_eur = z_95 * W_usd * nav_eur * sigma_1m

# VaR as % of NAV
fx_var_pct_nav = fx_var_eur / nav_eur * 100

In [50]:

# __LaTeX OUTPUT__

latex_template = Template(r"""
\subsubsection*{Exchange Rate Risk (EUR/USD)}

The portfolio has a net USD equity exposure of
\[
  W_{\text{USD}} = \frac{\text{USD equity value in EUR}}{\text{Total NAV in EUR}}
  = \frac{$usd_equity_value}{$nav_eur}
  = $W_usd
\]
which corresponds to $W_usd_pct\% of Net Asset Value (NAV).

Assuming an annualized EUR/USD volatility of
\[
  \sigma_{\text{FX,ann}} = $ann_vol\%,
\]
the corresponding one-month FX volatility is
\[
  \sigma_{\text{FX,1M}} = \sigma_{\text{FX,ann}} \cdot \sqrt{\frac{1}{12}} = $sigma_1m\%.
\]

Using a parametric Value-at-Risk (VaR) approach with a 95\% confidence level,
\[
  \text{VaR}_{0.95}^{1\text{M}} \approx z_{0.95} \cdot W_{\text{USD}} \cdot \text{NAV} \cdot \sigma_{\text{FX,1M}},
\]
where \( z_{0.95} = $z_value \) is the 95\% quantile of the standard normal distribution.

For the current portfolio this gives
\[
  \text{VaR}_{0.95}^{1\text{M}} \approx $var_eur\,\text{EUR},
\]
which corresponds to $var_pct\% of total NAV.
""")

latex_snippet = latex_template.substitute(
    usd_equity_value=f"{usd_equity_value_eur:,.2f}",
    nav_eur=f"{nav_eur:,.2f}",
    W_usd=f"{W_usd:.4f}",
    W_usd_pct=f"{W_usd*100:.2f}",
    ann_vol=f"{ann_fx_vol*100:.2f}",
    sigma_1m=f"{sigma_1m*100:.2f}",
    z_value=f"{z_95:.3f}",
    var_eur=f"{fx_var_eur:,.0f}",
    var_pct=f"{fx_var_pct_nav:.2f}"
)

In [51]:
# __UTPUT__

print("=== FX RISK SNAPSHOT ===")
print(f"Total NAV (EUR):           €{nav_eur:,.2f}")
print(f"USD equity value (EUR):    €{usd_equity_value_eur:,.2f}")
print(f"Net USD exposure W_USD:    {W_usd*100:.2f}% of NAV")
print()
print(f"Annualized FX vol:         {ann_fx_vol*100:.2f}%")
print(f"1M FX vol (log):           {sigma_1m*100:.2f}%")
print()
print(f"1M FX VaR @95%:            €{fx_var_eur:,.0f}  ({fx_var_pct_nav:.2f}% of NAV)")

=== FX RISK SNAPSHOT ===
Total NAV (EUR):           €7,551.24
USD equity value (EUR):    €3,762.18
Net USD exposure W_USD:    49.82% of NAV

Annualized FX vol:         5.00%
1M FX vol (log):           1.44%

1M FX VaR @95%:            €89  (1.18% of NAV)


### Interpretation

**Total NAV (EUR): €7,848.35**  
This is the **total value of the portfolio in EUR**, including all positions and cash.  
It’s the base against which all percentages (exposure, VaR as % of NAV, etc.) are measured.

---

**USD equity value (EUR): €4,012.90**  
This is the **EUR value of all USD-denominated equities** in the portfolio.  
It answers the question: *“If I convert all my USD stocks into EUR at today’s FX rate, how much are they worth?”*

---

**Net USD exposure \(W_USD\): 51.13% of NAV**  
This is the **share of the portfolio that is effectively exposed to USD**:

- Formula: `W_USD = USD equity value in EUR / Total NAV in EUR`
- Here, about **51% of your portfolio value moves with the USD/EUR exchange rate**.  
  If EUR strengthens vs USD, that half of the portfolio will generally lose value in EUR terms (and vice versa).

---

**Annualized FX vol: 5.00%**  
This is the **annualized volatility of EUR/USD**, based on historical data:

- Roughly: the **typical size of yearly percentage moves** in the FX rate (measured as standard deviation of returns).
- 5% means that, historically, the EUR/USD rate tends to fluctuate with a yearly standard deviation of about 5%.

---

**1M FX vol (log): 1.44%**  
This is the **1-month volatility of EUR/USD**, derived from the annualized vol:

- Computed as: `sigma_1m = 0.05 * sqrt(1/12) ≈ 0.0144` (i.e. 1.44%)
- Interpretation: over a **one-month horizon**, the FX rate typically moves by about **±1.44%** (one standard deviation), based on history.

---

**1M FX VaR @95%: €95 (1.21% of NAV)**  
This is the **1-month FX Value-at-Risk at 95% confidence**, in EUR and as % of NAV.

- Conceptually: *“Given our USD exposure and past FX volatility, how big could an FX loss be over one month, in a typical worst-case 5% scenario?”*
- Here:
  - In money terms: about **€95** potential FX loss.
  - As a share of the portfolio: about **1.21% of NAV**.

So, you can read this line as:

> “Due purely to EUR/USD movements, there is a 5% chance that the FX component of the portfolio loses **more than €95** over the next month, and a 95% chance that the FX loss will be **less than €95**.”


In [52]:
print("\n=== LATEX VERSION ===")
print(latex_snippet)


=== LATEX VERSION ===

\subsubsection*{Exchange Rate Risk (EUR/USD)}

The portfolio has a net USD equity exposure of
\[
  W_{\text{USD}} = \frac{\text{USD equity value in EUR}}{\text{Total NAV in EUR}}
  = \frac{3,762.18}{7,551.24}
  = 0.4982
\]
which corresponds to 49.82\% of Net Asset Value (NAV).

Assuming an annualized EUR/USD volatility of
\[
  \sigma_{\text{FX,ann}} = 5.00\%,
\]
the corresponding one-month FX volatility is
\[
  \sigma_{\text{FX,1M}} = \sigma_{\text{FX,ann}} \cdot \sqrt{\frac{1}{12}} = 1.44\%.
\]

Using a parametric Value-at-Risk (VaR) approach with a 95\% confidence level,
\[
  \text{VaR}_{0.95}^{1\text{M}} \approx z_{0.95} \cdot W_{\text{USD}} \cdot \text{NAV} \cdot \sigma_{\text{FX,1M}},
\]
where \( z_{0.95} = 1.645 \) is the 95\% quantile of the standard normal distribution.

For the current portfolio this gives
\[
  \text{VaR}_{0.95}^{1\text{M}} \approx 89\,\text{EUR},
\]
which corresponds to 1.18\% of total NAV.

